# Optimisation

Often the code we initiially write to solve a problem is primarily focused on accuracy but often once this is achieved we would like to find ways to make the code run faster.  Python isn't the best code to do this but we will we will use it to investigate the process for optimising code.  We will revisit this when we learn cython.

In order to understand how to make code faster it can be useful to know the basics of how a computer works. Most computers have an architecture that looks a bit like this:

![](Plots/Computer_architecture.png)

Modern CPU's are so fast that actually most optimisatising for high performance computing comes from two approaches: 1. optimising memory usage to make sure the CPU is fed with enough data 2. utilising parallelisation which we will meet next.  There are three numbers listed for each section in the above plot: Latency, which is how long it takes to respond to a command; Bandwidth, which is how fast it can pass data up the chain; Size, which is how much data can be stored there.  The main problem we see is that memory latency is ~200 times longer than CPU latency (which is 1 cycle).  So, if you want to multiply two numbers it takes 200 times as long to get them to the CPU as it does to multiply them.  The response to this was to introduce multiple layers of faster and faster, but smaller and smaller, memory inbetween the CPU and main memory called 'cache'.  This helps keep data that we are working on close to the CPU so if we want to use it we can without delay.  This is a problem that is getting worse with time as we can see in the following:

![](Plots/cpuvsmem.png)

This means that there are two things we can do to speed up our code.

- Keep data local, ie near the CPU, and 
- reuse it as much as possible while it is there.

Obviously you have no direct control over the cache but there are some things you can do that help the computer optimise this.

The best analogy for a computer is trying to learn something from textbooks.  Now the heirarchy looks like this:

![](Plots/Library.png)

And it is easy to get some basic lessons

- Get all the books you need from the library at once because it's a pain to keep going back there
- Constantly swapping books is annoying so try to finish working on the open ones before you look at the next set

With this in mind lets look at what we can do to optimise our code.

Firstly, in python we don't actually have much control over what the computer does at the instruction level.  This is a shame as often it is possible to speed up some calculations by orders of magnitude just by using simple directives to compilers in lower level languages like C and Fortran.  This is why it is so important to use packages in python where this type of optimisation should already have been done for you.  Another key point is that you should try to use packages what have been build for your CPU (check this with `lscpu`), for example making sure you install the intel distribution for intel chips with:

`conda create -n idp intelpython3_full python=3`

### Timing operations

There are always many different ways to do the same calculation so you should get into the habit of checking which is fastest.  For simple commands and functions we can just use `%timeit` (single line) and `%%timeit` (entire cell) which are useful for comparing equvilent code.  The best bit is they run the code multiple times to give you an accurate estimate of time.  Here are some examples of simple small speedups you can do:


In [ ]:
# 1. Squaring vs multiplication vs adding
x = 0.5
%timeit x**2
%timeit x*x
%timeit x/x

%timeit x+x
%timeit 2.0*x
%timeit 2.0*5.0
%timeit x/0.5

Strictly speaking on a CPU addition is faster than multiplication which is faster than division.  Exponentiation is not a basic operation for the CPU and instead calls a function which takes the log then multiplies.  However, in python we see that while exponentiation is slower than multiplying, multipling is the same as adding.  Multiplying by a number is faster than multiplying by a variable as we don't have to go and find the number it in memory and multipling two numbers much faster again.  We can roughly conclude that it takes twice as long to find a variable as it does to multiply it (you can read faster than find the right bit of a book). In python division and muliplication take the same time.  This is because most of the time is lost in finding the variables rather than the operation itself.  But if we do the operation alot:

In [ ]:
%timeit [0.5*x for x in range(10000)]
%timeit [x/2.0 for x in range(10000)]

Then small differences can emerge.

In [ ]:
# 2. Square root
import math
%timeit x**0.5
%timeit math.sqrt(x)

Python functions can be slower than basic operations.   I think here `sqrt` most likely has been tuned for higher accuracy.

In [ ]:
# 3a. Math vs Numpy
import math
import numpy as np
%timeit math.sin(x)
%timeit np.sin(x)

So numpy is slower for scalars but...

In [ ]:
# 3b. Math vs Numpy for vector
X = np.random.random(100)
Y1 = np.zeros(100)
Y2 = np.zeros(100)
Y3 = np.zeros(100)
%timeit Y1 = np.sin(X)
%timeit Y2 = list(map(lambda x: math.sin(x),X))

In [ ]:
%%timeit
for i in range(100):
    Y3[i] = math.sin(X[i]) 

faster for arrays, note the tiny scaling which shows that almost all the time is getting the function itself so again most of the time is in finding things, not calculation.  Here we have used `map` which is a way of applying a function to a list with the format:

map(function,list) ie:  `list(map(lambda x: x**2, items)`

There is also `filter` which selects based on a condition:

filter(function,list) ie: `list(filter(lambda x: x < 0, items))`

These are faster than loops but can't compete with numpy.   Here is something similar for matrix multiplication:

In [ ]:
# 4a. Math vs Numpy multiplication
x=2.3
y=3.4
%timeit np.dot(x,y)
%timeit x*y

In [ ]:
# 4b. Math vs Numpy for dot product
X = np.random.random(100)
Y = np.random.random(100)
%timeit np.dot(X,Y)
%timeit sum(list(map(lambda x:x[0]*x[1],list(zip(X,Y)))))

Now the call time for numpy is all the time and much faster than `map` construct

In [ ]:
# 5. Constructing lists
%timeit [x*x for x in range(100)]
%timeit [x**2 for x in range(100)]

In [ ]:
%%timeit
list1=[]
for x in range(100):
    list1.append(x*x)

In [ ]:
%%timeit
list1=[]
append = list1.append
for x in range(100):
    append(x*x)

In [ ]:
%%timeit
list1 = np.zeros((100))
for x in range(100):
    list1[x] = x*x

In [ ]:
%%timeit
np.fromfunction(lambda i,:i*i,(100,),dtype=int)

So list comprehensions are fastest. Also note that using `append = list1.append` saves us about 20% in time.  This is due to not having to looking up the function, the assignment has make it more 'local' in memory.  Do be careful with this as it can make code very hard to read so you should generally aviod it.  Also note that subsequent operations on list1 may be quicker for the last 2 methods as the list is more likely to be contiguous in memory (all in the same place) for very large lists so we can read it to cache faster.   Finally, note that using a list comprehension won't make you code fast if you use bad algorithms like `x**2` rather than `x*x` which costs ~5 times as much.  In general better algorithms will always win over better code.  This is why prototyping is such a good idea.

In [ ]:
#7. Different indexing orders
import numpy as np
A = np.zeros((100,100))
B = np.random.random((100,100))

In [ ]:
%%timeit
for i in range(100):
    for j in range(100):
        A[i,j] = B[i,j]

In [ ]:
%%timeit
for i in range(100):
    for j in range(100):
        A[j,i] = B[j,i]

In C or Fortran this would have mattered alot, but here the loops are so slow you don't see any difference due to how you are accessing memory.  This could still be something to try for very large arrays as it may eventually win.  The problem is that python stores arrays as a single line of all the rows in order so the `(i,j)` loop reads the data in order, so the next value is just next to the last one.  The `(j,i)` has to jump the length of the row to find the next value slowing it down.

In [ ]:
#8. Move stuff out of loops if possible
import math

def func1():
    x=math.sqrt(2)
    y=0
    for i in range(100):
        y*=x
        
def func2():
    y=0
    for i in range(100):
        y*=math.sqrt(2)
               
def func3():
    y=0
    for i in range(100):
        y*=1.414213
        
%timeit func1()
%timeit func2()
%timeit func3()

There is no point in working out the square root of 2 100 times. Again try to make stuff used in loops 'local' and avoid any calculation if possible (this doesn't happen for for loops which only expand the `for` line once, `while` does it every iteration)

In [ ]:
#8a. Move stuff out of loops if possible
import math
list1 = [range(100)]

def func1():
    y=0
    i=0
    while i<len(list1):
        y+=i
        i+=1
        
def func2():
    y=0
    i=0
    length = len(list1)
    while i<length:
        y+=i
        i+=1
        
%timeit func1()
%timeit func2()

Same goes for test conditons for while loops.

In [ ]:
#9. imports inside Vs outside of funtions

import math
def func1():
    math.sin(0.3)
    
def func2():
    import math
    math.sin(0.3)
    
    
%timeit func1()
%timeit func2()

It can be tempting to use imports inside functions so it looks tidy but this has significant cost. Best leave them at the top of the module as functions may be used multiple times but modules are (generally) only loaded once.

In [ ]:
#10. Function overhead

def func1(i):
    return i*i

def func2():
    x = 0e0
    for i in range(100):
        x += func1(i)
        
def func3():
    x = 0e0
    for i in range(100):
        x += i*i
        
%timeit func2()
%timeit func3()

Modulisation is great but function calls can be expensive so don't go crazy (especially in loops!).  Again this is introducing an extra lookup which makes the data less likely to be local in cache

In [ ]:
#11. Multiple assignment rather than tempory variables

def func1():
    a=0
    b=1
    for i in range(1000):
        a,b = b,a+b
        
def func2():
    a=0
    b=1
    for i in range(1000):
        c = a+b
        a = b
        b = c
        
%timeit func1()
%timeit func2()

A small saving (plus looks better)

In [ ]:
# 12. Finding elements depends on the data type
list1 = [range(100)]
set1 = set(list1)

%timeit 5 in list1
%timeit 5 in set1

Sets and dictionaries are hash tables so are faster to search than lists. 

In [ ]:
# 13. Read entire files at once if possible
import numpy as np
def read1():
    data1 = np.loadtxt('Data/Period1.txt')
    return data1

def read2():
    file = open('Data/Period1.txt','r')
    data1 = []
    for line in file:
        tmp = line.split()
        data1.append(float(tmp[0]))
    data2 = np.array(data1)
    return data2

def read3():
    file = open('Data/Period1.txt','r')
    data1 = file.read()
    data1 = data1.split('\n')
    data1 = [float(x) for x in data1[:-1]]
    data1 = np.array(data1)
    return data1

%timeit read1()
%timeit read2()
%timeit read3()

So numpy is the cleanest to code but the slowest to run and reading the file in one block is better than line by line (remember getting all library books at once rather than making multiple trips).  This is especially true on shared systems where you may compete for bandwidth for I/O.  There the system may alternate read statements from competing programmes eg: someone reads 1Tb, you read a line, someone reads 1Tb, you read one line etc.. which can be crippling.

All of the above will only give you small gains but can be worth considering.  Real gains almost always come finding better methods as:

- Algorithm improvement virtually always beats optimisation

If your code is slow you should first think about "how else I could try to calculate this?" rather than trying to find faster ways to do the same approach.
Don't be tempted do do any of these if it sacrifices code readability.  There is a famous quote amongst computer scientsts:

<b>"Premature optimisation is the root of all evil"</b>

Your first goal should always be clarity (luckly in python clarity often coincides with speed).

## Algorithmic Complexity

When selecting algorithms you should think about how they scale with the size of the problem.  This is the computational or algorithimic complexity.  There are seveal main groups: `linear`, `log-linear`, `polynomial` and `exponential`. Unlike the above where simplicity makes the code fast, usually the simplest algorithm will scale the worst.  A common example is sorting.  The simplest algorithim is `selection sort` which works like this:

1. Iteritivly find the ith smallest element and swap it's positon with the element at postion i

This is fairly simple and the way most people would think about sorting.  It is an $O(n^2)$ algorithm as we have to find the smallest of the remaining objects, which is $O(n)$, $O(n)$ times.  Code to do it is fairly simple:  

In [ ]:
def selection_sort(list1):        
    for i in range(len(list1)):
        minimum = i
        
        for j in range(i + 1, len(list1)):
            # Select the smallest value
            if list1[j] < list1[minimum]:
                minimum = j

        # Switch the element with the one at position i 
        list1[minimum], list1[i] = list1[i], list1[minimum]
            
    return list1


        
list1 = [7,5,7,9,5,3,5,4,8]  

print(selection_sort(list1))

However much faster algorithms exist which scale much better.  If we use a divide an conquer approach this complexity can be reduced to $O(nlog(n))$.  This can be seen in the example of `merge sort` which works using the fact that merging sorted lists is $O(n)$.  Our algorithm looks like this:

1. Make $n$ lists 1 element long.
2. Merge list in pairs to make $n/2$ sorted lists, 2 elements long
3. Itererate until only one sorted list remains.

Here is the code to do it, it's much more complicated now:

In [ ]:
def merge_sort(arr):
    # The last array split
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    # Perform merge_sort recursively on both halves
    left, right = merge_sort(arr[:mid]), merge_sort(arr[mid:])

    # Merge each side together
    return merge(left, right, arr.copy())


def merge(left, right, merged):

    left_cursor, right_cursor = 0, 0
    while left_cursor < len(left) and right_cursor < len(right):
      
        # Sort each one and place into the result
        if left[left_cursor] <= right[right_cursor]:
            merged[left_cursor+right_cursor]=left[left_cursor]
            left_cursor += 1
        else:
            merged[left_cursor + right_cursor] = right[right_cursor]
            right_cursor += 1
            
    for left_cursor in range(left_cursor, len(left)):
        merged[left_cursor + right_cursor] = left[left_cursor]
        
    for right_cursor in range(right_cursor, len(right)):
        merged[left_cursor + right_cursor] = right[right_cursor]

    return merged

list1 = [7,5,7,9,5,3,5,4,8]  

print(merge_sort(list1))

In [ ]:
import numpy as np
list1 = np.random.normal(0,5,(10))
list2 = np.random.normal(0,5,(10))

%timeit selection_sort(list1)
%timeit merge_sort(list2)

list1 = np.random.normal(0,5,(1000))
list2 = np.random.normal(0,5,(1000))

%timeit selection_sort(list1)
%timeit merge_sort(list2)

This type of optimisation is tricky and requires a bit of clever thinking.  For standard operations it is work serching a bit to check if there is a fancy way of doing it (and a package that implements it!) but for specilised cases you are best to try asking your local computer officers or computer science graduates.  

The key to optimising your code is to note that usually in any code it's only 10% (or less!) of it that takes 90% of the time. So optimising 90% of the code will have almost no effect on runtime. This means we should initially always write for clarity then try to find which parts ate taking most of the time.

How do we find the 10% of our code to focus on?  The answer is to **profile** your code.


### Profiliing

Profilers analyse your code and tell you what parts are taking the most time (and memory) to run.  There are a few you can use.  Let's look at them with our little prime number generator we created

In [ ]:
def primenum(N):
    primes = [2]
    for n in range(3,N):
        if all(n%p>0 for p in primes):
            primes.append(n)
    return primes 

%prun primenum(10000)

This indicates that we spent most of our time in `<genexp>` on line 4 with the next most time spent on the built-in method `all`.  Which is what we would expect

Next we could install the `line_profiler` package (`conda install line_profiler`) and use this instead

In [ ]:
%load_ext line_profiler

def primenum(N):
    primes = [2]
    for n in range(3,N):
        if all(n%p>0 for p in primes):
            primes.append(n)
    return primes 


%lprun -f primenum primenum(10000)

Which is a little easier to read. Here is an intersting case to see how memory reuse is important

In [ ]:
%load_ext line_profiler

import math
import numpy as np

def nbody(particles):
    for item1 in particles:
        acc1 = 0
        acc2 = 0
        acc3 = 0
        for item2 in particles:
            dx = item1[0] - item2[0]
            dy = item1[1] - item2[1]
            dz = item1[2] - item2[2]
            
            distsquare = dx*dx+dy*dy+dz*dz
            if abs(distsquare) > 1e-9:
                distinv = 1e0/math.sqrt(distsquare)
                acc1 += dx * distinv*distinv*distinv
                acc2 += dy * distinv*distinv*distinv
                acc3 += dz * distinv*distinv*distinv

particles = np.random.random((100,3))

%lprun -f nbody nbody(particles)

So why do the 2nd and 3rd lines get faster for the blocks of three commands next to each other?  It's because the first moves the data to l1 cache so it's easier to use in the next line.

As one of our main problems is likely to be memory useage we can also profile memory with the `memory_profiler` package (`conda install memory_profiler`) but this only works on files so we need to save this to a file first

In [ ]:
%%file Tools/primetools.py
def primenum(N):
    primes = [2]
    for n in range(3,N):
        if all(n%p>0 for p in primes):
            primes.append(n)
    return primes

In [ ]:
#%load_ext memory_profiler
import sys
sys.path.append('./Tools') 
import primetools as pts 
%mprun -f pts.primenum pts.primenum(10000)

We can also use `%run -p` to profile scripts that we run.  Finally there is also another build in profiler `cProfile`

In [ ]:
import cProfile
def primenum(N):
    primes = [2]
    for n in range(3,N):
        if all(n%p>0 for p in primes):
            primes.append(n)
    return primes 

cProfile.run('primenum(10000)')

This can also be run from the command line with `python -m cProfile [-o output_file] [-s sort_order] myscript.py` which is usefull

### Exercise:

Profile both your periodic signal code from lecture 6 and your 'game of life' code from 7 and see if you can get them to run any faster. Try the same with you previous solutions from earlier lectures.  You should use this in your protoyping stage to check the quality of modules you may want to use.